# Principal Component Analysis

### Authors

| Name | Roll Number |
|-|-|
| Gautam Singh | CS21BTECH11018 |
| Jaswanth Beere | BM21BTECH11007 |

This `.ipynb` notebook performs Principal Component Analysis (PCA) on the SVHN
dataset.

## Obtaining the Data

The SVHN dataset is obtained using the `curl` command on the appropriate file
URL.

In [ ]:
%%bash

if [ ! -f train_32x32.mat ]; then
    curl -O http://ufldl.stanford.edu/housenumbers/train_32x32.mat
fi

In [ ]:
%%bash

if [ ! -f test_32x32.mat ]
then
    curl -O http://ufldl.stanford.edu/housenumbers/test_32x32.mat
fi

## Package Imports

For this exercise, the following packages must be installed and imported.

In [ ]:
%pip install numpy scikit-learn scipy matplotlib

In [ ]:
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from scipy import io
from sklearn.neighbors import KNeighborsClassifier

## Data Visualization

We begin by visualizing part of the raw data. First, we must load the data into
`numpy` arrays for processing.

In [ ]:
# Load training and test data

train_data = io.loadmat('train_32x32.mat')
test_data = io.loadmat('test_32x32.mat')

# Separate training data inputs (images) and outputs (classes)

train_X = train_data['X']
train_Y = train_data['y']
test_X = test_data['X']
test_Y = test_data['y']

NUM_TRAIN_SAMPLES = train_X.shape[-1]
NUM_TEST_SAMPLES = test_X.shape[-1]

Next, we reshape the dimensions of `train_X` such that the first index
represents sample number. Further, we scale all features to zero mean and unit
variance.

In [ ]:
# Reshape train_X such that train_X[i] is the ith image in the dataset
# Further, flatten image arrays for PCA.

train_X_reshaped = train_X.reshape(32*32*3, NUM_TRAIN_SAMPLES).T
test_X_reshaped = test_X.reshape(32*32*3, NUM_TEST_SAMPLES).T
train_Y_reshaped = train_Y.ravel()
test_Y_reshaped = test_Y.ravel()

## Sampling the Data

To perform PCA on the given dataset, we sample 2500 data points of each digit
from 0 to 9, giving us 25000 training examples to perform PCA on. Note that 0 is
represented as 10 in the dataset.

In [ ]:
# Sample 2500 training examples for each digit
NUM_SAMPLES_PER_CLASS = 2500

# Use a numpy random number generator
rng = np.random.default_rng()

# Array of randomly chosen indices
ind_arr = np.array([])

for i in range(1,11,1):
    # Find the indices in the training set corresponding to digit
    idx = np.where(train_Y == i)[0]
    # Generate samples randomly
    rand_samples = rng.choice(idx, NUM_SAMPLES_PER_CLASS, False)
    # Append to index array
    ind_arr = np.append(ind_arr, rand_samples)

ind_arr = np.asarray(ind_arr, int)
# Finally, generate random samples and their labels
train_X_gen = train_X_reshaped[ind_arr]
train_Y_gen = train_Y[ind_arr]

We are now in a position to perform PCA on our training set and visualize how
the proportion of variance (PoV) changes with the number of top eigenvectors
considered.

In [ ]:
# Perform PCA of the generated training set using only the top 10 eigenvectors
pca = PCA(n_components=10)
train_X_pca = pca.fit_transform(train_X_gen)

# Perform PCA transform of the test data
test_X_pca = pca.transform(test_X_reshaped)

Here, we visualize the top 10 eigenvectors and their eigenvalues. Then, we
provide a reconstrcution of a sample from each class using these top 10
eigenvectors.

In [ ]:
# Number of PCA components
n_components = pca.n_components_
# Visualize top 10 eigenvectors and eigenvalues
print(pca.components_)
print(pca.singular_values_)

In [ ]:
# Visualize the reconstructed samples
fig = plt.figure()
# Note that each class has a starting index of 2500*i
for i in range(10):
    # Add subplot
    sub = fig.add_subplot(2,5,i+1)
    # Perform inverse pca transform
    sub.imshow(np.asarray(pca.inverse_transform(train_X_pca[NUM_SAMPLES_PER_CLASS*i]), int).reshape(32,32,3))
    sub.title.set_text('Digit ' + str((i+1)%10))

fig.suptitle('Visualization of Images Using Top 10 Eigenvectors')
plt.tight_layout()

Clearly, the images are not reconstructed clearly, as very few (10) eigenvectors
are used compared to the dimension (3072) of the images in the PCA.

## PoV Against the Number of Eigenvectors

To find the ideal number of eigenvectors to use in the PCA, we plot the
Proportion of Variance (PoV) as a function of the number of eigenvectors
considered. We set our threshold PoV to be 0.9 _i.e._, a 90% reconstruction of
the image is ideal.

In [ ]:
# Perform PCA with ALL dimensions. This can take a while.
full_pca = PCA()
train_X_allpca = full_pca.fit(train_X_gen)

In [ ]:
# PoV array (zero padded at the start)
cumulative_variance = np.zeros(len(full_pca.explained_variance_ratio_) + 1)
cumulative_variance[1:] = np.cumsum(full_pca.explained_variance_ratio_)

# Plot the cumulative variance as well as cutoff 0.9 PoV
plt.plot(cumulative_variance)
plt.plot(np.arange(cumulative_variance.shape[0]), 0.9*np.ones_like(cumulative_variance), '--')
plt.xlabel('Number of Eigenvectors')
plt.ylabel('PoV')
plt.title('Proportion of Variance (PoV) Against Number of Eigenvectors')
plt.legend(['PoV of Data', 'Required PoV (0.9)'])
plt.grid()
plt.tight_layout()
plt.show()

From the above figure, it is quite evident that the top 10 eigenvectors will be
insufficient. To find the ideal number of eigenvectors, we can perform binary
search on the `cumulative_variance` array.

In [ ]:
pca_comp = np.searchsorted(cumulative_variance, 0.9)
pca_comp

It turns out that the ideal number of components needed for the PCA is 27.

## k-NN and PCA

Now, we perform the k-nearest neighbours classification algorithm on the
training dataset, with and without PCA. We also demonstrate the difference in
test accuracy on using different number of eigenvectors for the PCA as found in
the previous sections. Here, we consider `k = 5, 7`.

In [ ]:
# Define kNN classifier objects
neigh_five = KNeighborsClassifier(n_neighbors=5)
neigh_seven = KNeighborsClassifier(n_neighbors=7)

### k-NN on Raw Data

The raw data obtained is directly run with kNN, and the accuracies are reported
below.

In [ ]:
# Run kNN with k=5 and k=7. This might take a while.

# Using k=5
neigh_five = neigh_five.fit(train_X_gen, train_Y_gen.ravel())
neigh_five_score_raw = neigh_five.score(test_X_reshaped, test_Y_reshaped)
# Using k=7
neigh_seven = neigh_seven.fit(train_X_gen, train_Y_gen.ravel())
neigh_seven_score_raw = neigh_seven.score(test_X_reshaped, test_Y_reshaped)
# Report accuracy
print('Accuracy for raw data using k = 5:', neigh_five_score_raw)
print('Accuracy for raw data using k = 7:', neigh_seven_score_raw)

### k-NN on PCA Preprocessed Data

To improve the accuracy (which is about 41.86%) of kNN, we perform it on the
preprocessed data.

#### Using 10 Eigenvectors

In [ ]:
# Run kNN with k=5 and k=7. This might take a while.

# Using k=5
neigh_five = neigh_five.fit(train_X_pca, train_Y_gen.ravel())
neigh_five_score_raw = neigh_five.score(test_X_pca, test_Y_reshaped)
# Using k=7
neigh_seven = neigh_seven.fit(train_X_pca, train_Y_gen.ravel())
neigh_seven_score_raw = neigh_seven.score(test_X_pca, test_Y_reshaped)
# Report accuracy
print('Accuracy for raw data using k = 5:', neigh_five_score_raw)
print('Accuracy for raw data using k = 7:', neigh_seven_score_raw)

#### Using PoV = 0.9